# Top features per dimension

In [1]:
import sys 
import os 

sys.path.append('../..')

DATA_DIR = '../../data'
os.environ['DATA_DIR'] = DATA_DIR

from copy import deepcopy
from utils.data import load_data, load_dimension_embeddings, load_sorting
from utils.correlation import vectorize_concepts
from utils.dimensions import calc_top_feature_per_dim, matrix_to_top_list

import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from scipy.spatial.distance import squareform
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

nlp = spacy.load("en_core_web_sm")

2022-11-04 12:08:13.044648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-04 12:08:13.044693: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
min_amount_runs_feature_occured = 5
group_to_one_concept = True
min_amount_runs_feature_occured_within_concept = 1
run_nr = None 
duplicates = True 
gpt_df, mc_df, behv_sim, cslb_df, sorting_df, _ = load_data(True, True, min_amount_runs_feature_occured, min_amount_runs_feature_occured_within_concept, group_to_one_concept, run_nr, duplicates)

../../data


/home/hannes/anaconda3/envs/features/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Feature-Concept Matrix with TF-IDF 

normalized across concepts, so that features like is small are weighted less as they are present in almost all concepts

In [3]:
gpt_vec_count = vectorize_concepts(gpt_df, load_sorting(), 'bla', 'tfidf')


/home/hannes/anaconda3/envs/features/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Feature-Concept Matrix with Counts

normalized by dividing by the number of concepts the features occurs

In [ ]:
gpt_vec_binary = vectorize_concepts(gpt_df, load_sorting(), 'bla', 'binary')

def count_concepts(values):
    n = len([value for value in values if value != 0])
    return n

counts = gpt_vec_binary.groupby(lambda x: True).agg(count_concepts).reset_index(drop=True).T

gpt_vec_count = vectorize_concepts(gpt_df, load_sorting(), 'bla', 'count')


,0
absorbs water,4
amplifies sound,4
attracts iron,1
attracts metal,1
attracts nails,1
...,...
winks clothes,4
winks dishes,1
writes,3
writes on paper,6


In [ ]:


# when CSLB is used we can only use the overlap concepts
#dimension_embeddings = dimension_embeddings.loc[dimension_embeddings.index.isin(intersection_concepts)]

list(gpt_vec_binary.index) == list(dimension_embeddings.index)

True

In [ ]:
#def weight_feature_counts_with_dimension_values(df, dimension_values, normalized):
#    df = df.mul(dimension_values, axis=0)
#    df = df.sum(axis=0).to_frame()

#    if normalized:
#        df = df.div(counts)
        
#    df = df.reset_index() 
#    df = df.rename(columns={df.columns[0]: 'feature', df.columns[1]: 'weight'})  

# Load dimension weights 

In [4]:
n_dims = 49
dimension_embeddings = load_dimension_embeddings(n_dims)
dims = dimension_embeddings.columns

# norm sum = 1 per dimension -> needed for normalization by dimensions
dim_sums = dimension_embeddings.sum(axis=0)
print(dim_sums.shape)
dimension_embeddings = dimension_embeddings.div(dim_sums)
print(dimension_embeddings.shape)


(49,)
(1854, 49)


In [ ]:
df=pd.DataFrame({'Dimension': load_dimension_embeddings(49).columns}).reset_index()
df['index'] = df['index'] + 1
print(df.to_latex(index=False))

\begin{tabular}{rl}
\toprule
 index &                                        Dimension \\
\midrule
     1 &                made of metal / artificial / hard \\
     2 &  food-related / eating-related / kitchen-related \\
     3 &                         animal-related / organic \\
     4 &             clothing-related / fabric / covering \\
     5 & furniture-related / household-related / artifact \\
     6 &                            plant-related / green \\
     7 &                                 outdoors-related \\
     8 &             transportation / motorized / dynamic \\
     9 &                          wood-related / brownish \\
    10 &                                body part-related \\
    11 &                                         colorful \\
    12 &              valuable / special occasion-related \\
    13 &                          electronic / technology \\
    14 &    sport-related / recreational activity-related \\
    15 &                              disc-shap

In [5]:
normed_features = calc_top_feature_per_dim(dimension_embeddings, gpt_vec_count, dims)

normed_features.to_csv(f'./normed_features_per_{n_dims}_dimension_matrix.csv')
matrix_to_top_list(normed_features).to_csv(f'./normed_features_per_{n_dims}_dimension_list.csv', index=False)

Run dimension: made of metal / artificial / hard
Run dimension: food-related / eating-related / kitchen-related
Run dimension: animal-related / organic
Run dimension: clothing-related / fabric / covering
Run dimension: furniture-related / household-related / artifact
Run dimension: plant-related / green
Run dimension: outdoors-related
Run dimension: transportation / motorized / dynamic
Run dimension: wood-related / brownish
Run dimension: body part-related
Run dimension: colorful
Run dimension: valuable / special occasion-related
Run dimension: electronic / technology
Run dimension: sport-related / recreational activity-related
Run dimension: disc-shaped / round
Run dimension: tool-related
Run dimension: many small things / course pattern
Run dimension: paper-related / thin / flat / text-related
Run dimension: fluid-related / drink-related
Run dimension: long / thin
Run dimension: water-related / blue
Run dimension: powdery / fine-scale pattern
Run dimension: red
Run dimension: feminin